In [1]:
import sys
import os
import itertools
sys.path.append(os.path.abspath('../'))
from state_constrainer_YY import state_constrainer
from IPython.utils import io
import time

In [2]:
m = state_constrainer('boundedStackInvariant.sol')

# -- Deploying Contract
(level _is_sat_z3_call) took 0.06341290473937988 seconds (sat)
# -- Contract Deployed      (took 31.513577222824097 seconds)


In [3]:
metodos = ["push","pop"]
precondiciones = [x+"_precondition" for x in metodos]
def call_pre():
    for cond in precondiciones:
        m.callContractFunction(cond,tx_sender=m.witness_account)

In [4]:
states = list(itertools.product([0,1],repeat=len(metodos)))
def repr_state(state):
    text = ""
    for x,method in zip(state,metodos):
        text += '_'+method if x else ""
    if text == "":
        text = "vacio"
    return text

In [5]:
call_pre()

In [6]:
found = []
with io.capture_output() as captured:    
    for state in states:
        count = m.generateTestCases(keys=precondiciones,
                                    targets=state,
                                    testcaseName=f"STATE_{repr_state(state)}")
        if count > 0:
            found.append(f"found {count} testcases that reach {repr_state(state)} initial state")
        else:
            found.append(f"found no testcases for {repr_state(state)} initial state")

In [7]:
for line in found:
    print(line)

found 1 testcases that reach vacio initial state
found no testcases for _pop initial state
found 1 testcases that reach _push initial state
found no testcases for _push_pop initial state


In [8]:
m.take_snapshot()
with io.capture_output() as captured:    
    m.callContractFunction("push")
call_pre()

(level _is_sat_z3_call) took 0.005105257034301758 seconds (sat)
(level _is_sat_z3_call) took 0.015276670455932617 seconds (sat)


In [9]:
def repr_transition(state1,state2,metodo):
    return f"{repr_state(state1)}->{metodo}->{repr_state(state2)}"

In [10]:
ini_state = (1,0) #_push
found = []
with io.capture_output() as captured:    
    for fin_state in states:
        count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                    targets=(ini_state+fin_state),
                                    testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'push')}")
        if count > 0:
            found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'push')}")
        else:
            found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'push')}")

In [11]:
for line in found:
    print(line)

found no testcases for transition _push->push->vacio
found 1 testcases for transition _push->push->_pop
found no testcases for transition _push->push->_push
found 1 testcases for transition _push->push->_push_pop


In [ ]:
m.take_snapshot()
start_setter = time.time()
with io.capture_output() as captured:
    m.callContractFunction("setter")
    call_pre()
print(time.time()-start_setter)

In [ ]:
m.take_snapshot()
m.callContractFUnction("push")
call_pre()

In [ ]:
ini_state = (1,1) #_push_pop
found = []
with io.capture_output() as captured:    
    for fin_state in states:
        count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                    targets=(ini_state+fin_state),
                                    testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'push')}")
        if count > 0:
            found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'push')}")
        else:
            found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'pus

In [ ]:
for line in found:
    print(line)

In [ ]:
m.goto_snapshot()
m.take_snapshot()
m.callContractFUnction("pop")
call_pre()

In [ ]:
ini_states = [(1,1),(0,1)] #_push_pop , #_pop
found = []

for ini_state in ini_states:
    for fin_state in states:
        with io.capture_output() as captured:    
            count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                        targets=(ini_state+fin_state),
                                        testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'pop')}")
            if count > 0:
                found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'pop')}")
            else:
                found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'pop')}")

In [ ]:
for line in found:
    print(line)